In [ ]:
import pandas as pd
import numpy as np
rng = np.random.default_rng(42)

## Series

Given the Series below, without entering the statements:

In [ ]:
s = pd.Series(np.arange(5),index=list("abcde"))

- predict the values and the type of object returned for each statement:

In [ ]:
s['d']        # value at index 'd'
s['b':'d']    # values from indices 'b', 'c' and 'd'
s[2::2][::-1] # values from the second position to the end in steps of 2, then reversed
s[['b', 'a']] # values of sets of indices ['b', 'a']

- predict the contents of `s`, `s1` and `lst`:

In [ ]:
# 'lst' is a ndarray and Series 's' has a reference to 'lst' as its content, also termed a view. Any
# change made to 's' will be a change to 'lst'. However, 's1' is a copy of 's' therefore any change
# to 's1' is not propagated to 's' nor 'lst'.
#
lst, idx = np.arange(5), list("abcde")
s = pd.Series(lst,idx)
s[-1:] = 10               # -1: is equivalent to -1 which marks the last position and is updated to 10 in 's' and 'lst'
lst[0] = 5                # changes is 'lst' will be reflected in 's', therefore s[0] is equal to 5
s1 = pd.Series(s.copy())  # s1 is a copy of s, a separate memory location
s1[0] = -1                # s1[0] is set to -1 and s[0]=lst[0]=5

- predict the result of operation

In [ ]:
# Arithmetic operations on Series is done according to the overlapping indices and
# the symmetric difference is set to NaN.
s1 = pd.Series({'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4})
s2 = pd.Series({'d': 0, 'e': 1, 'f': 2, 'g': 3})

s1 + s2           # only 'd' and 'e' are common and are summed
s1[3:] * s2[:-2]  # slices pick  overlapping indices: [3,4] * [0,1] = [0,4]

## DataFrame

Give the de dataframe below:

retrieve:
- row 2 as a Series as wel as a dataFrame
- rows on even positions
- rows with even indices
- 3d column
- odd (index) rows and columns 'b' to 'd'

In [ ]:
rng = default_rng(1234)
df = pd.DataFrame(np.array(rng.standard_normal(25)).reshape(5,5),
             index=[1, 0, 4, 3, 2], columns=list("abcde"))

In [ ]:
type(df.loc[2])                   # row 2 as Series
type(df.loc[[2]])                 # row 2 as DataFrame
df.loc[df.index[0::2]]            # rows on even positions
df.loc[df.index % 2 ==0]          # rows with even indices
df.iloc[:,2]                      # 3rd column <=> df['c']
df.loc[df.index % 2 !=0,'b':'d']  # odd index rows of columns 'b' and 'd'

### Merge DataFrames

Given `df1`, `df2` and `df3` apply the following:

- merge df1 and df2 side by side
- merge df1 and df3 stacked
- merge all and reset index

In [ ]:
df1 = pd.DataFrame({'name': ['ants', 'bees','wasps'] , 'order':['Hymenoptera']*3})
df2 = pd.DataFrame({'name': ['beetles', 'weevils'] , 'order':['Coleoptera']*2})
df3 = pd.DataFrame({'name': ['butterflies', 'moths'], 'order':['Lepidoptera']*2 })

In [ ]:
pd.concat([df1, df2], axis=1)
pd.concat([df1, df3], axis=0)
pd.concat([df1, df2, df3], axis=0,ignore_index=True) #

### Missing values

Given the following DataFrame

In [ ]:
df = pd.DataFrame(np.arange(25).reshape(5,5))

set the values to NaN to reproduce the following

In [ ]:
df.loc[0,0::2] = np.nan
df.loc[::2,2] = np.nan
df.loc[:,4] = np.nan
df.loc[2,:] = np.nan

Apply the following on the dataframe with missing values created in the previous step.

Drop missing:
- rows with missing values
- columns with missing values
- rows where all values are missing
- columns where all values are missing

Fill missing:
- with 0
- with mean based on column values
- with median based on row values

In [ ]:
# drop
df.dropna(axis=0)             # all rows have missing values
df.dropna(axis=1)             # all columns have missing values
df.dropna(axis=0, how='all')  # row 2 has all missing
df.dropna(axis=1, how='all')  # column 4 has all missing
# fill with 0
df.fillna(0)

# Vertical axis, axis=0, is taken to get the column means. Note that fillna has
# axis=0
#
df.fillna(df.mean(axis=0))

# Horizontal axis, axis=1, is taken to get the row means. However, pandas currently
# does not broadcast along the rows and df.fillna(df.mean(axis=1), axis=1)  will
# lead to NotImplementedError. Therefore, it needs to be implemented.
#
# 1) row-by-row
df.apply(lambda row: row.fillna(row.mean()), axis=1)
# 2) transpose
df.T.fillna(df.T.mean()).T

### Natural gas consumption in the Netherlands

The dataset can be downloaded from [CBS Open data StatLine](https://opendata.cbs.nl/statline/portal.html?_la=en&_catalog=CBS). A version is already included in the data directory of this session's git repository.

We first read the data with `pd.read_csv`. For this exercise we only work with the columns `Periods` and `TotalSupply_1:

In [ ]:
cbs = pd.read_csv("data/00372eng_UntypedDataSet_16032025_171431.csv",sep=";")
df = cbs[['Periods','TotalSupply_1']]

Each entry give the total consumption `TotaalAanbod_1` in units of million cubic metre (MCM). The column `Periods`has the year (yyyy) followed by a tag {JJ,KW,MM} representing the yearly, quarterly and monthly terms respectively, followed by two digits `00..12`. The two digit followed by the tags have different meaning per tag. For JJ it is always `00`, MM with `00..12` for 12 months and `KW` with  `01..04` for four quarters.

In order to get more control over the entry date we will need to create a new variable `date` in the dataset using the given `Periods`. Hint: see Series.str accessor functions for manipulating strings and pd.Timestamp for dates.

In [ ]:
df = df.Periods.str.split(r'(JJ|MM|KW)', regex=True, expand=True)  # expand=True forces the result into
                                                                    # a DataFrame
df = pd.DataFrame({'year': df[0].astype(int),                       # Create DataFrame {year,term,idx}
                    'term': df[1],
                    'idx': df[2].astype(int)})
df = pd.concat([df,cbs[['TotalSupply_1']]],axis=1)

1. Write a function given a Series with {year,term,idx} returns a timestamp according to the specification below. In all cases the last day of the period is chosen as the date, e.g. 1954KW03  will be 30-09-1954.

```
JJ : yyyyJJ00 => 31-12-yyyy
KW : yyyyKWmm => where mm in {1,2,3,4}
                 01: 1-1-yyyy to 31-3-yyyy
                 02: 1-4-yyyy to 30-6-yyyy
                 03: 1-7-yyyy to 30-9-yyyy
                 04: 1-10-yyyy to 31-12-yyyy
MM : yyyyMMmm => dd-mm-yyyy where dd is the last day of the month and
                 mm in {1,..,12}
```


2. Create a new DataFrame called `ngc` (natural gas consumption) with three columns {term, date, consumption} :
    - term : {JJ,KW,MM}
    - date : timestamps
    - consumption: which is `TotalSupply_1` only renamed

In [ ]:
def to_ts(s):
    """
    Apply the following timestamp format for each term {JJ,KW,MM}:

    JJ : yyyyJJ00 => 31-12-yyyy
    KW : yyyyKWdd => 01: 1-1-yyyy to 31-3-yyyy
                     02: 1-4-yyyy to 30-6-yyyy
                     03: 1-7-yyyy to 30-9-yyyy
                     04: 1-10-yyyy to 31-12-yyyy
    MM : yyyyMMdd => 1-xx-yyyy

    :param format_:
    :param s: {year,term,idx}
    :return: Timestamp
    """
    year_, term, idx = s

    #
    # import sys
    # sys.version

    if term == 'JJ':
        day_, month_ = 31, 12
        return pd.Timestamp(year=year_, month=month_, day=day_)
    elif term == 'KW':
        day_, month_ = [(31,3),(30,6),(30,9),(31,12)][idx-1]
        return pd.Timestamp(year=year_, month=month_, day=day_)
    elif term == 'MM':
        day_, month_ = pd.Timestamp(year=year_, month=idx, day=1).days_in_month, idx
        return pd.Timestamp(year=year_, month=month_, day=day_)
    else:
        raise Exception('invalid tag, valid tags:  {JJ, KW, MM} !')
#
# _main_
#

cbs = pd.read_csv("data/00372eng_UntypedDataSet_16032025_171431.csv",sep=";")
df = cbs[['Periods']].copy()
df = df.Periods.str.split(r'(JJ|MM|KW)', regex=True, expand=True)      # Split 'Period' into {year,term,index}.
df = pd.DataFrame({'year': df[0].astype(int),                          # Create DataFrame {year,term,idx} with
                   'term': df[1],                                      # the correct type.
                   'idx' : df[2].astype(int)})
consumption = cbs[['TotalSupply_1']].rename(columns={'TotalSupply_1': 'consumption'}) #
df['date'] = [to_ts(s) for _,s in df.iterrows()]                       # create the date variable
ngc  = pd.concat([df[['term', 'date']],consumption], axis=1)           # combine {term, date, consumption}

3. Evaluate the entries in `ngc` dataframe by checking whether the sum of monthly 'MM' and quarterly 'KW' add up to yearly 'JJ' consumptions.

In [ ]:
jj,kw,mm = [ngc.loc[ngc.term==t] for t in ['JJ','KW','MM']]
j = jj.groupby(jj.date.dt.year)['consumption'].sum()
q = kw.groupby(kw.date.dt.year)['consumption'].sum()
m = mm.groupby(mm.date.dt.year)['consumption'].sum()
# check only years present in all three categories, KW has the least years 1982.. 2022
(m.loc[q.index] == q).value_counts()
(j.loc[q.index] == q).value_counts()

In [ ]:
jj,kw,mm = [ngc.loc[ngc.term==t] for t in ['JJ','KW','MM']]
# count total consumption for {JJ,KW,MM}
j = jj.groupby(jj.date.dt.year)['consumption'].sum()
q = kw.groupby(kw.date.dt.year)['consumption'].sum()
m = mm.groupby(mm.date.dt.year)['consumption'].sum()

# Check only years present in all three categories,
# KW has the least years 1982.. 2022, therefore only
# those years are compared.
(m.loc[q.index] == q).value_counts() # i) compare MM and KW consumptions correspond?
(j.loc[q.index] == q).value_counts() # ii) What about JJ and KW?